In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.style as style
import seaborn as sns

# from collections import deque
# from scipy.stats import linregress
# from scipy.optimize import curve_fit
# import math
# from scipy import stats

import logging
import traceback
import json

import utils.step_utils as su
import utils.file_utils as fu
import utils.config_utils as cu
import utils.step_init as step_init
import step_control

In [2]:
EXP_NAME = 'data'
SAVE_PATH = os.getcwd()
EXP_DIR = os.path.join(SAVE_PATH, EXP_NAME)
PUMP_CAL_PATH = os.path.join(SAVE_PATH, 'pump_cal.json')

EXCEL_CONFIG_FILE = "experiment_configurations.xlsx"
vials = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]

In [3]:
logger = logging.getLogger(__name__)
log_name = os.path.join(EXP_DIR, 'evolver.log')
def setup_logging(filename, quiet, verbose):
    if quiet:
        logging.basicConfig(level=logging.CRITICAL + 10)
    else:
        if verbose == 0:
            level = logging.INFO
        elif verbose >= 1:
            level = logging.DEBUG
        logging.basicConfig(format='%(asctime)s - %(name)s - [%(levelname)s] '
                            '- %(message)s',
                            datefmt='%Y-%m-%d %H:%M:%S',
                            filename=filename,
                            level=level)
setup_logging(log_name, False, False)

class EvolverNamespace():
    start_time = None
    use_blank = False
    OD_initial = None
    experiment_params = None
    ip_address = None
    exp_dir = EXP_DIR

    def stop_exp(self):
        "STOPPED EXPERIMENT"
    
    def get_flow_rate(self):
        pump_cal = None
        with open(PUMP_CAL_PATH) as f:
            pump_cal = json.load(f)
        return pump_cal['coefficients']
    
    def calc_growth_rate(self, vial, gr_start, elapsed_time):
        ODfile_name =  "vial{0}_OD.txt".format(vial)
        # Grab Data and make setpoint
        OD_path = os.path.join(EXP_DIR, 'OD', ODfile_name)
        OD_data = np.genfromtxt(OD_path, delimiter=',')
        raw_time = OD_data[:, 0]
        raw_OD = OD_data[:, 1]
        raw_time = raw_time[np.isfinite(raw_OD)]
        raw_OD = raw_OD[np.isfinite(raw_OD)]

        # Trim points prior to gr_start
        trim_time = raw_time[np.nonzero(np.where(raw_time > gr_start, 1, 0))]
        trim_OD = raw_OD[np.nonzero(np.where(raw_time > gr_start, 1, 0))]

        # Take natural log, calculate slope
        log_OD = np.log(trim_OD)
        slope, intercept, r_value, p_value, std_err = linregress(
            trim_time[np.isfinite(log_OD)],
            log_OD[np.isfinite(log_OD)])
        logger.debug('growth rate for vial %s: %.2f' % (vial, slope))

        # Save slope to file
        file_name =  "vial{0}_gr.txt".format(vial)
        gr_path = os.path.join(EXP_DIR, 'growthrate', file_name)
        text_file = open(gr_path, "a+")
        text_file.write("{0},{1}\n".format(elapsed_time, slope))
        text_file.close()

eVOLVER = EvolverNamespace()

In [4]:
configs = cu.load_excel_configs(EXCEL_CONFIG_FILE)
elapsed_time = 300000
# step_init.validate_selection_parameters(configs['selection-step_generation'], ) # Validate the selection parameters TODO: add logger and make this work
step_init.update_selection_configs(elapsed_time, vials, configs, logger, eVOLVER)
# step_init.plot_steps(vials, 'selection-steps', 'Selection', eVOLVER.exp_dir)

In [5]:
fu.labeled_last_n_lines('selection-control', 2, 1, EXP_DIR).iloc[0].stock_concentration#.convert_dtypes().dtypes

'40'

In [6]:
fu.get_last_n_lines('growthrate', 2, 3 * 2, EXP_DIR)

Unable to open file: c:\Users\nb\Dropbox\PC\Desktop\share\git\stepped-dpu\incremental_changes\stepped-dpu\experiment\template\data\growthrate\vial2_growthrate.txt
	Error: [Errno 2] No such file or directory: 'c:\\Users\\nb\\Dropbox\\PC\\Desktop\\share\\git\\stepped-dpu\\incremental_changes\\stepped-dpu\\experiment\\template\\data\\growthrate\\vial2_growthrate.txt'


array([], dtype=float64)

In [7]:
def multi_vial_control(vials, exp_dir, dilution_window, elapsed_time, logger, eVOLVER, time_out, VOLUME, lower_thresh, flow_rate, bolus_slow):
    """
    Controls multiple vials using the SteppedController class.

    Parameters:
    - vials: List of vial numbers to control.
    - exp_dir: Path to the experiment directory.
    - dilution_window: Number of data points for dilution calculations.
    - elapsed_time: Time since the start of the experiment (in hours).
    - logger: Logger instance for logging events and actions.
    - eVOLVER: eVOLVER instance for hardware interaction.

    Returns:
    - MESSAGE: Combined fluidics command for all vials.
    """
    MESSAGE = ['--'] * 48  # Initialize the MESSAGE array for fluidics commands

    for vial in vials:
        try:
            # Create a SteppedController instance for the current vial
            controller = step_control.SteppedController(vial, exp_dir, dilution_window, logger, elapsed_time, eVOLVER)

            # Perform control operations for this vial
            MESSAGE = controller.control(MESSAGE, time_out, VOLUME, lower_thresh, flow_rate, bolus_slow)

        except Exception as e:
            logger.error(f"Error controlling selection in vial {vial}: {e}")

    return MESSAGE

# Example setup
vials = [2]  # List of vials to control
dilution_window = 3
elapsed_time = 470  # Replace with actual elapsed time

# Fluidics parameters
time_out = 5
VOLUME = 20
lower_thresh = 1.6
flow_rate = eVOLVER.get_flow_rate()
bolus_slow = 0.1

# Call the multi-vial control function
MESSAGE = multi_vial_control(vials, EXP_DIR, dilution_window, elapsed_time, logger, eVOLVER, time_out, VOLUME, lower_thresh, flow_rate, bolus_slow)
print("Fluidics Message:", MESSAGE)

Fluidics Message: ['--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--']


In [8]:
import time

def time_function(iterations, func, *args, **kwargs):
    """
    Measure the time it takes to run a function a specified number of times.

    Parameters:
    - iterations: Number of times to run the function.
    - func: The function to be timed.
    - *args: Positional arguments to pass to the function.
    - **kwargs: Keyword arguments to pass to the function.

    Returns:
    - elapsed_time: Total time in seconds it took to run the function the specified number of times.
    - time_per_iteration: Average time per iteration in seconds.
    """
    start_time = time.time()
    for _ in range(iterations):
        func(*args, **kwargs)
    end_time = time.time()
    elapsed_time = end_time - start_time
    time_per_iteration = elapsed_time / iterations
    return elapsed_time, time_per_iteration

# Example usage:
# Define a sample function to be timed
def sample_function(x):
    return x ** 2

# Measure the time it takes to run sample_function 1000 times
iterations = 1000
elapsed_time, time_per_iteration = time_function(iterations, sample_function, 10)
print(f"Total time taken to run the function {iterations} times: {elapsed_time} seconds")
print(f"Average time per iteration: {time_per_iteration} seconds")

Total time taken to run the function 1000 times: 0.0 seconds
Average time per iteration: 0.0 seconds
